<a href="https://colab.research.google.com/github/jiteshpabla/Meal-Detection-using-CGM-data-/blob/main/Meal_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [ ]:
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt

In [ ]:
df_cgm = pd.read_csv("CGMData.csv")
df_cgm = df_cgm.dropna(axis='columns', how="all")
df_cgm

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (32) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Index,Date,Time,Sensor Glucose (mg/dL),ISIG Value,Event Marker,Sensor Exception
0,20355,2/12/2018,13:22:27,118.0,15.16,NaN,NaN
1,20356,2/12/2018,13:17:27,122.0,16.12,NaN,NaN
2,20357,2/12/2018,13:12:27,NaN,NaN,NaN,SENSOR_ERROR
3,20358,2/12/2018,13:07:27,NaN,NaN,NaN,SENSOR_ERROR
4,20359,2/12/2018,13:02:27,NaN,NaN,NaN,SENSOR_ERROR
...,...,...,...,...,...,...,...
55338,52855,7/25/2017,12:28:54,311.0,44.91,NaN,NaN
55339,52856,7/25/2017,12:23:54,311.0,44.26,NaN,NaN
55340,52857,7/25/2017,12:18:54,309.0,44.51,NaN,NaN
55341,52858,7/25/2017,12:13:54,310.0,45.16,NaN,NaN


In [ ]:
df_insulin = pd.read_csv("InsulinData.csv")
df_insulin = df_insulin.dropna(axis='columns', how="all")
df_insulin

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (13,43) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Index,Date,Time,New Device Time,BG Reading (mg/dL),Basal Rate (U/h),Bolus Type,Bolus Volume Selected (U),Bolus Volume Delivered (U),Bolus Duration (h:mm:ss),Prime Type,Prime Volume Delivered (U),Alarm,Suspend,Rewind,BWZ Estimate (U),BWZ Target High BG (mg/dL),BWZ Target Low BG (mg/dL),BWZ Carb Ratio (g/U),BWZ Insulin Sensitivity (mg/dL/U),BWZ Carb Input (grams),BWZ BG Input (mg/dL),BWZ Correction Estimate (U),BWZ Food Estimate (U),BWZ Active Insulin (U),Sensor Calibration BG (mg/dL),Bolus Number,Bolus Cancellation Reason,BWZ Unabsorbed Insulin Total (U),Final Bolus Estimate,Scroll Step Size,Insulin Action Curve Time,Sensor Calibration Rejected Reason,Bolus Source,Network Device Associated Reason,Network Device Disassociated Reason,Network Device Disconnected Reason
0,0,2/12/2018,13:20:53,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TEMPORARY,NaN,NaN
1,1,2/12/2018,13:20:48,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Other Alarm Code: 112,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2/12/2018,13:18:48,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,2/12/2018,13:18:48,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,USER_SUSPEND,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,2/12/2018,13:12:33,NaN,NaN,NaN,Normal,0.03,0.03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,120.0,NaN,NaN,NaN,NaN,NaN,NaN,CLOSED_LOOP_MICRO_BOLUS,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41430,22135,7/24/2017,19:00:01,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
41431,22136,7/24/2017,18:59:44,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
41432,22137,7/24/2017,18:59:44,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NORMAL_PUMPING,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
41433,22138,7/24/2017,18:59:44,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Rewind,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_cgm2 = pd.read_excel("CGMData670GPatient3.xlsx")
df_cgm2 = df_cgm2.dropna(axis='columns', how="all")
df_cgm2

,Index,Date,Time,Sensor Glucose (mg/dL),ISIG Value,Sensor Exception
0,21012,2018-03-07,11:30:48,172.0,26.04,NaN
1,21013,2018-03-07,11:25:48,172.0,26.12,NaN
2,21014,2018-03-07,11:20:48,172.0,26.48,NaN
3,21015,2018-03-07,11:15:48,170.0,25.91,NaN
4,21016,2018-03-07,11:10:48,176.0,26.23,NaN
...,...,...,...,...,...,...
33243,43244,2017-09-05,11:06:21,226.0,37.04,NaN
33244,43245,2017-09-05,11:01:21,214.0,34.65,NaN
33245,43246,2017-09-05,10:56:21,201.0,33.32,NaN
33246,43247,2017-09-05,10:51:21,183.0,32.09,NaN


In [ ]:
df_insulin2 = pd.read_excel("InsulinAndMealIntake670GPatient3.xlsx")
df_insulin2 = df_insulin2.dropna(axis='columns', how="all")
df_insulin2

,Index,Date,Time,New Device Time,BG Reading (mg/dL),Basal Rate (U/h),Temp Basal Amount,Temp Basal Type,Temp Basal Duration (h:mm:ss),Bolus Type,Bolus Volume Selected (U),Bolus Volume Delivered (U),Prime Type,Prime Volume Delivered (U),Alarm,Suspend,Rewind,BWZ Estimate (U),BWZ Target High BG (mg/dL),BWZ Target Low BG (mg/dL),BWZ Carb Ratio (g/U),BWZ Insulin Sensitivity (mg/dL/U),BWZ Carb Input (grams),BWZ BG Input (mg/dL),BWZ Correction Estimate (U),BWZ Food Estimate (U),BWZ Active Insulin (U),Sensor Calibration BG (mg/dL),Bolus Number,Bolus Cancellation Reason,BWZ Unabsorbed Insulin Total (U),Final Bolus Estimate,Scroll Step Size,Insulin Action Curve Time,Sensor Calibration Rejected Reason,Bolus Source,Network Device Associated Reason,Network Device Disassociated Reason,Network Device Disconnected Reason
0,6564,2018-03-07,11:36:15,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TEMPORARY,NaN,NaN
1,6565,2018-03-07,11:35:56,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Other Alarm Code: 112,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,6566,2018-03-07,11:33:19,NaT,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,6567,2018-03-07,11:33:19,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,USER_SUSPEND,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,6568,2018-03-07,11:31:04,NaT,NaN,NaN,NaN,NaN,NaN,Normal,0.05,0.05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,221.0,NaN,NaN,NaN,NaN,NaN,NaN,CLOSED_LOOP_MICRO_BOLUS,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23180,17311,2017-09-05,08:26:00,NaT,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23181,17312,2017-09-05,08:25:35,NaT,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23182,17313,2017-09-05,08:25:35,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NORMAL_PUMPING,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23183,17314,2017-09-05,08:25:35,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Rewind,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Convert to data time

In [ ]:
def convert_datetime(df, only_date=False):
    df['Date']= pd.to_datetime(df['Date']) 
    if not only_date:
        df['Time']= pd.to_datetime(df['Time'])
    return

In [ ]:
convert_datetime(df_insulin)
convert_datetime(df_insulin2, True)
convert_datetime(df_cgm2, True)
convert_datetime(df_cgm)

In [ ]:
def add_timestamp(df, timeCol_has_no_date = False): #true for df_cgm2
  df["timestamp"] = None
  i = 0
  for idx, row in df.iterrows():
    #print(row["Date"], "   ", row["Time"])
    #print(str(row["Date"])[:-9]+str(row["Time"])[-9:])
    if not timeCol_has_no_date:
      df.at[idx,'timestamp'] = pd.Timestamp(str(row["Date"])[:-9]+str(row["Time"])[-9:])
    else:
      df.at[idx,'timestamp'] = pd.Timestamp(str(row["Date"])[:-9]+" "+str(row["Time"]))
  #df["timestamp"] = "aaaa"

In [ ]:
add_timestamp(df_cgm)
df_cgm

,Index,Date,Time,Sensor Glucose (mg/dL),ISIG Value,Event Marker,Sensor Exception,timestamp
0,20355,2018-02-12,2020-10-28 13:22:27,118.0,15.16,NaN,NaN,2018-02-12 13:22:27
1,20356,2018-02-12,2020-10-28 13:17:27,122.0,16.12,NaN,NaN,2018-02-12 13:17:27
2,20357,2018-02-12,2020-10-28 13:12:27,NaN,NaN,NaN,SENSOR_ERROR,2018-02-12 13:12:27
3,20358,2018-02-12,2020-10-28 13:07:27,NaN,NaN,NaN,SENSOR_ERROR,2018-02-12 13:07:27
4,20359,2018-02-12,2020-10-28 13:02:27,NaN,NaN,NaN,SENSOR_ERROR,2018-02-12 13:02:27
...,...,...,...,...,...,...,...,...
55338,52855,2017-07-25,2020-10-28 12:28:54,311.0,44.91,NaN,NaN,2017-07-25 12:28:54
55339,52856,2017-07-25,2020-10-28 12:23:54,311.0,44.26,NaN,NaN,2017-07-25 12:23:54
55340,52857,2017-07-25,2020-10-28 12:18:54,309.0,44.51,NaN,NaN,2017-07-25 12:18:54
55341,52858,2017-07-25,2020-10-28 12:13:54,310.0,45.16,NaN,NaN,2017-07-25 12:13:54


In [ ]:
add_timestamp(df_cgm2, True)
df_cgm2

,Index,Date,Time,Sensor Glucose (mg/dL),ISIG Value,Sensor Exception,timestamp
0,21012,2018-03-07,11:30:48,172.0,26.04,NaN,2018-03-07 11:30:48
1,21013,2018-03-07,11:25:48,172.0,26.12,NaN,2018-03-07 11:25:48
2,21014,2018-03-07,11:20:48,172.0,26.48,NaN,2018-03-07 11:20:48
3,21015,2018-03-07,11:15:48,170.0,25.91,NaN,2018-03-07 11:15:48
4,21016,2018-03-07,11:10:48,176.0,26.23,NaN,2018-03-07 11:10:48
...,...,...,...,...,...,...,...
33243,43244,2017-09-05,11:06:21,226.0,37.04,NaN,2017-09-05 11:06:21
33244,43245,2017-09-05,11:01:21,214.0,34.65,NaN,2017-09-05 11:01:21
33245,43246,2017-09-05,10:56:21,201.0,33.32,NaN,2017-09-05 10:56:21
33246,43247,2017-09-05,10:51:21,183.0,32.09,NaN,2017-09-05 10:51:21


In [ ]:
add_timestamp(df_insulin2, True)
add_timestamp(df_insulin)

In [ ]:
df_insulin

,Index,Date,Time,New Device Time,BG Reading (mg/dL),Basal Rate (U/h),Bolus Type,Bolus Volume Selected (U),Bolus Volume Delivered (U),Bolus Duration (h:mm:ss),Prime Type,Prime Volume Delivered (U),Alarm,Suspend,Rewind,BWZ Estimate (U),BWZ Target High BG (mg/dL),BWZ Target Low BG (mg/dL),BWZ Carb Ratio (g/U),BWZ Insulin Sensitivity (mg/dL/U),BWZ Carb Input (grams),BWZ BG Input (mg/dL),BWZ Correction Estimate (U),BWZ Food Estimate (U),BWZ Active Insulin (U),Sensor Calibration BG (mg/dL),Bolus Number,Bolus Cancellation Reason,BWZ Unabsorbed Insulin Total (U),Final Bolus Estimate,Scroll Step Size,Insulin Action Curve Time,Sensor Calibration Rejected Reason,Bolus Source,Network Device Associated Reason,Network Device Disassociated Reason,Network Device Disconnected Reason,timestamp
0,0,2018-02-12,2020-10-28 13:20:53,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TEMPORARY,NaN,NaN,2018-02-12 13:20:53
1,1,2018-02-12,2020-10-28 13:20:48,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Other Alarm Code: 112,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-02-12 13:20:48
2,2,2018-02-12,2020-10-28 13:18:48,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-02-12 13:18:48
3,3,2018-02-12,2020-10-28 13:18:48,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,USER_SUSPEND,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-02-12 13:18:48
4,4,2018-02-12,2020-10-28 13:12:33,NaN,NaN,NaN,Normal,0.03,0.03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,120.0,NaN,NaN,NaN,NaN,NaN,NaN,CLOSED_LOOP_MICRO_BOLUS,NaN,NaN,NaN,2018-02-12 13:12:33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41430,22135,2017-07-24,2020-10-28 19:00:01,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-07-24 19:00:01
41431,22136,2017-07-24,2020-10-28 18:59:44,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-07-24 18:59:44
41432,22137,2017-07-24,2020-10-28 18:59:44,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NORMAL_PUMPING,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-07-24 18:59:44
41433,22138,2017-07-24,2020-10-28 18:59:44,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Rewind,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-07-24 18:59:44


In [ ]:
df_insulin2

,Index,Date,Time,New Device Time,BG Reading (mg/dL),Basal Rate (U/h),Temp Basal Amount,Temp Basal Type,Temp Basal Duration (h:mm:ss),Bolus Type,Bolus Volume Selected (U),Bolus Volume Delivered (U),Prime Type,Prime Volume Delivered (U),Alarm,Suspend,Rewind,BWZ Estimate (U),BWZ Target High BG (mg/dL),BWZ Target Low BG (mg/dL),BWZ Carb Ratio (g/U),BWZ Insulin Sensitivity (mg/dL/U),BWZ Carb Input (grams),BWZ BG Input (mg/dL),BWZ Correction Estimate (U),BWZ Food Estimate (U),BWZ Active Insulin (U),Sensor Calibration BG (mg/dL),Bolus Number,Bolus Cancellation Reason,BWZ Unabsorbed Insulin Total (U),Final Bolus Estimate,Scroll Step Size,Insulin Action Curve Time,Sensor Calibration Rejected Reason,Bolus Source,Network Device Associated Reason,Network Device Disassociated Reason,Network Device Disconnected Reason,timestamp
0,6564,2018-03-07,11:36:15,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TEMPORARY,NaN,NaN,2018-03-07 11:36:15
1,6565,2018-03-07,11:35:56,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Other Alarm Code: 112,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-03-07 11:35:56
2,6566,2018-03-07,11:33:19,NaT,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-03-07 11:33:19
3,6567,2018-03-07,11:33:19,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,USER_SUSPEND,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-03-07 11:33:19
4,6568,2018-03-07,11:31:04,NaT,NaN,NaN,NaN,NaN,NaN,Normal,0.05,0.05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,221.0,NaN,NaN,NaN,NaN,NaN,NaN,CLOSED_LOOP_MICRO_BOLUS,NaN,NaN,NaN,2018-03-07 11:31:04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23180,17311,2017-09-05,08:26:00,NaT,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-09-05 08:26:00
23181,17312,2017-09-05,08:25:35,NaT,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-09-05 08:25:35
23182,17313,2017-09-05,08:25:35,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NORMAL_PUMPING,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-09-05 08:25:35
23183,17314,2017-09-05,08:25:35,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Rewind,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-09-05 08:25:35


In [ ]:
df_cgm2

,Index,Date,Time,Sensor Glucose (mg/dL),ISIG Value,Sensor Exception,timestamp
0,21012,2018-03-07,11:30:48,172.0,26.04,NaN,2018-03-07 11:30:48
1,21013,2018-03-07,11:25:48,172.0,26.12,NaN,2018-03-07 11:25:48
2,21014,2018-03-07,11:20:48,172.0,26.48,NaN,2018-03-07 11:20:48
3,21015,2018-03-07,11:15:48,170.0,25.91,NaN,2018-03-07 11:15:48
4,21016,2018-03-07,11:10:48,176.0,26.23,NaN,2018-03-07 11:10:48
...,...,...,...,...,...,...,...
33243,43244,2017-09-05,11:06:21,226.0,37.04,NaN,2017-09-05 11:06:21
33244,43245,2017-09-05,11:01:21,214.0,34.65,NaN,2017-09-05 11:01:21
33245,43246,2017-09-05,10:56:21,201.0,33.32,NaN,2017-09-05 10:56:21
33246,43247,2017-09-05,10:51:21,183.0,32.09,NaN,2017-09-05 10:51:21


# Get meal data

In [ ]:
from datetime import timedelta, datetime

In [ ]:
#get meal points
meal_point = df_insulin.query("`BWZ Carb Input (grams)` == `BWZ Carb Input (grams)`")
meal_point

,Index,Date,Time,New Device Time,BG Reading (mg/dL),Basal Rate (U/h),Bolus Type,Bolus Volume Selected (U),Bolus Volume Delivered (U),Bolus Duration (h:mm:ss),Prime Type,Prime Volume Delivered (U),Alarm,Suspend,Rewind,BWZ Estimate (U),BWZ Target High BG (mg/dL),BWZ Target Low BG (mg/dL),BWZ Carb Ratio (g/U),BWZ Insulin Sensitivity (mg/dL/U),BWZ Carb Input (grams),BWZ BG Input (mg/dL),BWZ Correction Estimate (U),BWZ Food Estimate (U),BWZ Active Insulin (U),Sensor Calibration BG (mg/dL),Bolus Number,Bolus Cancellation Reason,BWZ Unabsorbed Insulin Total (U),Final Bolus Estimate,Scroll Step Size,Insulin Action Curve Time,Sensor Calibration Rejected Reason,Bolus Source,Network Device Associated Reason,Network Device Disassociated Reason,Network Device Disconnected Reason,timestamp
48,48,2018-02-12,2020-10-28 09:15:45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.6,NaN,NaN,13.0,NaN,34.0,153.0,0.05,2.60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-02-12 09:15:45
129,129,2018-02-12,2020-10-28 02:30:55,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.2,NaN,NaN,12.0,NaN,15.0,0.0,0.00,1.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-02-12 02:30:55
149,149,2018-02-12,2020-10-28 00:15:41,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.5,NaN,NaN,12.0,NaN,0.0,290.0,2.55,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-02-12 00:15:41
188,188,2018-02-11,2020-10-28 20:33:18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.8,NaN,NaN,8.0,NaN,71.0,0.0,0.00,8.88,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-02-11 20:33:18
202,202,2018-02-11,2020-10-28 19:32:19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,115.0,80.0,8.0,100.0,0.0,143.0,0.20,0.00,0.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-02-11 19:32:19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41347,22052,2017-07-25,2020-10-28 18:31:40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.1,140.0,90.0,9.5,65.0,115.0,104.0,0.00,12.10,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-07-25 18:31:40
41371,22076,2017-07-25,2020-10-28 13:45:08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,140.0,90.0,8.5,65.0,0.0,0.0,0.00,0.00,2.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-07-25 13:45:08
41374,22079,2017-07-25,2020-10-28 13:25:21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,140.0,90.0,8.5,65.0,0.0,297.0,2.40,0.00,3.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-07-25 13:25:21
41393,22098,2017-07-25,2020-10-28 10:39:46,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.4,140.0,90.0,9.0,65.0,58.0,181.0,0.60,6.40,6.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-07-25 10:39:46


In [ ]:
meal_point2 = df_insulin2.query("`BWZ Carb Input (grams)` == `BWZ Carb Input (grams)`")
#meal_point2 = df_insulin2.query("`BWZ Carb Input (grams)` > 0")
meal_point2

,Index,Date,Time,New Device Time,BG Reading (mg/dL),Basal Rate (U/h),Temp Basal Amount,Temp Basal Type,Temp Basal Duration (h:mm:ss),Bolus Type,Bolus Volume Selected (U),Bolus Volume Delivered (U),Prime Type,Prime Volume Delivered (U),Alarm,Suspend,Rewind,BWZ Estimate (U),BWZ Target High BG (mg/dL),BWZ Target Low BG (mg/dL),BWZ Carb Ratio (g/U),BWZ Insulin Sensitivity (mg/dL/U),BWZ Carb Input (grams),BWZ BG Input (mg/dL),BWZ Correction Estimate (U),BWZ Food Estimate (U),BWZ Active Insulin (U),Sensor Calibration BG (mg/dL),Bolus Number,Bolus Cancellation Reason,BWZ Unabsorbed Insulin Total (U),Final Bolus Estimate,Scroll Step Size,Insulin Action Curve Time,Sensor Calibration Rejected Reason,Bolus Source,Network Device Associated Reason,Network Device Disassociated Reason,Network Device Disconnected Reason,timestamp
60,6624,2018-03-07,07:13:37,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.7,140.0,120.0,13.0,92.0,0.0,297.0,1.70,0.00,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-03-07 07:13:37
98,6662,2018-03-07,00:07:36,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.5,140.0,120.0,10.0,92.0,0.0,358.0,2.30,0.00,0.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-03-07 00:07:36
136,6700,2018-03-06,20:53:35,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.8,NaN,NaN,9.2,NaN,50.0,175.0,0.38,5.43,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-03-06 20:53:35
184,6748,2018-03-06,17:07:24,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,8.6,NaN,0.0,209.0,1.05,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-03-06 17:07:24
215,6779,2018-03-06,12:40:57,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.7,NaN,NaN,8.6,NaN,40.0,247.0,0.13,4.65,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-03-06 12:40:57
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23118,17249,2017-09-05,19:32:06,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.6,140.0,120.0,9.2,92.0,52.0,127.0,0.00,5.60,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-09-05 19:32:06
23121,17252,2017-09-05,18:55:41,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,140.0,120.0,9.2,92.0,0.0,128.0,0.00,0.00,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-09-05 18:55:41
23135,17266,2017-09-05,15:53:43,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,140.0,120.0,8.6,92.0,0.0,160.0,0.20,0.00,1.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-09-05 15:53:43
23148,17279,2017-09-05,13:14:52,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.4,140.0,120.0,8.6,92.0,38.0,247.0,1.10,4.40,0.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-09-05 13:14:52


In [ ]:
def remove_overlap(df):
  df_nooverlap = pd.DataFrame()
  for idx, row in df.iterrows():
    #if row["Date"] == pd.Timestamp("2017-07-25"):
    # adding buffer 5 mins to the time period; just to make sure
    df_test = df[ (df["timestamp"] > (row["timestamp"] + timedelta(hours=0, minutes=-36))) & (df["timestamp"] < (row["timestamp"] + timedelta(hours=2, minutes=6)))]
    if len(df_test) == 1:
      #print(row)
      df_nooverlap = df_nooverlap.append(df_test)
  return df_nooverlap

In [ ]:
meal_point_nooverlap = remove_overlap(meal_point)
meal_point_nooverlap

,Index,Date,Time,New Device Time,BG Reading (mg/dL),Basal Rate (U/h),Bolus Type,Bolus Volume Selected (U),Bolus Volume Delivered (U),Bolus Duration (h:mm:ss),Prime Type,Prime Volume Delivered (U),Alarm,Suspend,Rewind,BWZ Estimate (U),BWZ Target High BG (mg/dL),BWZ Target Low BG (mg/dL),BWZ Carb Ratio (g/U),BWZ Insulin Sensitivity (mg/dL/U),BWZ Carb Input (grams),BWZ BG Input (mg/dL),BWZ Correction Estimate (U),BWZ Food Estimate (U),BWZ Active Insulin (U),Sensor Calibration BG (mg/dL),Bolus Number,Bolus Cancellation Reason,BWZ Unabsorbed Insulin Total (U),Final Bolus Estimate,Scroll Step Size,Insulin Action Curve Time,Sensor Calibration Rejected Reason,Bolus Source,Network Device Associated Reason,Network Device Disassociated Reason,Network Device Disconnected Reason,timestamp
48,48,2018-02-12,2020-10-28 09:15:45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.6,NaN,NaN,13.0,NaN,34.0,153.0,0.05,2.60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-02-12 09:15:45
129,129,2018-02-12,2020-10-28 02:30:55,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.2,NaN,NaN,12.0,NaN,15.0,0.0,0.00,1.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-02-12 02:30:55
149,149,2018-02-12,2020-10-28 00:15:41,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.5,NaN,NaN,12.0,NaN,0.0,290.0,2.55,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-02-12 00:15:41
188,188,2018-02-11,2020-10-28 20:33:18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.8,NaN,NaN,8.0,NaN,71.0,0.0,0.00,8.88,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-02-11 20:33:18
261,261,2018-02-11,2020-10-28 12:43:23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.2,NaN,NaN,13.0,NaN,27.0,220.0,1.20,2.08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-02-11 12:43:23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41164,21869,2017-07-27,2020-10-28 09:19:41,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.2,140.0,90.0,9.0,65.0,47.0,129.0,0.00,5.20,0.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-07-27 09:19:41
41172,21877,2017-07-27,2020-10-28 05:45:51,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.8,140.0,90.0,11.0,80.0,24.0,201.0,0.70,2.10,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-07-27 05:45:51
41214,21919,2017-07-26,2020-10-28 19:15:06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.2,140.0,90.0,9.5,65.0,60.0,86.0,-0.10,6.30,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-07-26 19:15:06
41261,21966,2017-07-26,2020-10-28 12:48:41,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.4,140.0,90.0,8.5,65.0,63.0,255.0,1.70,7.40,2.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-07-26 12:48:41


In [ ]:
meal_point2_nooverlap = remove_overlap(meal_point2)
meal_point2_nooverlap

,Index,Date,Time,New Device Time,BG Reading (mg/dL),Basal Rate (U/h),Temp Basal Amount,Temp Basal Type,Temp Basal Duration (h:mm:ss),Bolus Type,Bolus Volume Selected (U),Bolus Volume Delivered (U),Prime Type,Prime Volume Delivered (U),Alarm,Suspend,Rewind,BWZ Estimate (U),BWZ Target High BG (mg/dL),BWZ Target Low BG (mg/dL),BWZ Carb Ratio (g/U),BWZ Insulin Sensitivity (mg/dL/U),BWZ Carb Input (grams),BWZ BG Input (mg/dL),BWZ Correction Estimate (U),BWZ Food Estimate (U),BWZ Active Insulin (U),Sensor Calibration BG (mg/dL),Bolus Number,Bolus Cancellation Reason,BWZ Unabsorbed Insulin Total (U),Final Bolus Estimate,Scroll Step Size,Insulin Action Curve Time,Sensor Calibration Rejected Reason,Bolus Source,Network Device Associated Reason,Network Device Disassociated Reason,Network Device Disconnected Reason,timestamp
60,6624,2018-03-07,07:13:37,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.7,140.0,120.0,13.0,92.0,0.0,297.0,1.70,0.00,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-03-07 07:13:37
98,6662,2018-03-07,00:07:36,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.5,140.0,120.0,10.0,92.0,0.0,358.0,2.30,0.00,0.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-03-07 00:07:36
136,6700,2018-03-06,20:53:35,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.8,NaN,NaN,9.2,NaN,50.0,175.0,0.38,5.43,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-03-06 20:53:35
184,6748,2018-03-06,17:07:24,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,8.6,NaN,0.0,209.0,1.05,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-03-06 17:07:24
215,6779,2018-03-06,12:40:57,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.7,NaN,NaN,8.6,NaN,40.0,247.0,0.13,4.65,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-03-06 12:40:57
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23076,17207,2017-09-06,08:48:28,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.1,140.0,120.0,13.0,92.0,60.0,191.0,0.50,4.60,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-09-06 08:48:28
23105,17236,2017-09-05,22:12:32,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,140.0,120.0,9.2,92.0,0.0,193.0,0.50,0.00,1.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-09-05 22:12:32
23135,17266,2017-09-05,15:53:43,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,140.0,120.0,8.6,92.0,0.0,160.0,0.20,0.00,1.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-09-05 15:53:43
23148,17279,2017-09-05,13:14:52,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.4,140.0,120.0,8.6,92.0,38.0,247.0,1.10,4.40,0.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-09-05 13:14:52


In [ ]:
def extract_meal_data(df_meal, df):
  meal_data = pd.DataFrame()
  i = 0
  for idx, row in df_meal.iterrows():
    df_test = df[ (df["timestamp"] > (row["timestamp"] + timedelta(hours=0, minutes=-30))) & (df["timestamp"] < (row["timestamp"] + timedelta(hours=2, minutes=0)))]
    #print(len(df_test))
    if len(df_test) != 30:
      #print(len(df_test), row["timestamp"])
      continue
    meal_data = meal_data.append([df_test["Sensor Glucose (mg/dL)"].to_list()])
    i += 1
  print(i)
  return meal_data

In [ ]:
meal_data1 = extract_meal_data(meal_point_nooverlap, df_cgm)
meal_data1.dropna(inplace=True)
meal_data1

606


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
0,150.0,162.0,174.0,180.0,178.0,184.0,185.0,194.0,193.0,186.0,182.0,174.0,172.0,167.0,166.0,164.0,165.0,166.0,167.0,164.0,159.0,160.0,158.0,158.0,153.0,154.0,151.0,150.0,149.0,150.0
0,69.0,80.0,89.0,98.0,100.0,111.0,123.0,139.0,160.0,178.0,186.0,185.0,186.0,191.0,201.0,228.0,242.0,263.0,277.0,284.0,287.0,288.0,289.0,279.0,260.0,262.0,259.0,250.0,245.0,250.0
0,177.0,194.0,200.0,201.0,189.0,162.0,166.0,173.0,176.0,169.0,167.0,166.0,168.0,180.0,182.0,178.0,171.0,160.0,145.0,138.0,132.0,120.0,127.0,129.0,132.0,123.0,104.0,100.0,107.0,106.0
0,208.0,215.0,212.0,206.0,203.0,196.0,190.0,180.0,184.0,180.0,177.0,186.0,194.0,205.0,207.0,220.0,200.0,194.0,196.0,198.0,200.0,201.0,211.0,215.0,217.0,216.0,221.0,221.0,219.0,209.0
0,202.0,197.0,191.0,185.0,197.0,206.0,209.0,207.0,174.0,168.0,168.0,142.0,136.0,135.0,136.0,147.0,148.0,137.0,137.0,144.0,141.0,135.0,133.0,126.0,110.0,104.0,74.0,62.0,58.0,57.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,200.0,198.0,191.0,183.0,175.0,172.0,157.0,147.0,140.0,132.0,127.0,129.0,137.0,131.0,127.0,120.0,112.0,108.0,99.0,87.0,77.0,70.0,58.0,55.0,57.0,57.0,53.0,46.0,40.0,40.0
0,152.0,153.0,165.0,173.0,177.0,179.0,183.0,189.0,196.0,200.0,204.0,213.0,211.0,212.0,203.0,192.0,179.0,169.0,162.0,149.0,130.0,123.0,122.0,125.0,124.0,129.0,133.0,137.0,141.0,145.0
0,209.0,210.0,209.0,210.0,210.0,213.0,216.0,212.0,213.0,210.0,205.0,203.0,192.0,179.0,172.0,169.0,168.0,170.0,176.0,181.0,183.0,188.0,194.0,201.0,201.0,199.0,200.0,204.0,210.0,212.0
0,81.0,77.0,74.0,67.0,70.0,72.0,74.0,75.0,71.0,67.0,66.0,69.0,77.0,89.0,112.0,123.0,132.0,130.0,125.0,121.0,112.0,100.0,87.0,83.0,71.0,60.0,40.0,40.0,40.0,40.0


In [ ]:
meal_data2 = extract_meal_data(meal_point2_nooverlap, df_cgm2)
meal_data2.dropna(inplace=True)
meal_data2

446


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
0,77.0,74.0,76.0,80.0,86.0,98.0,112.0,128.0,142.0,151.0,165.0,196.0,205.0,208.0,239.0,273.0,283.0,289.0,294.0,301.0,299.0,295.0,291.0,286.0,273.0,270.0,252.0,246.0,246.0,248.0
0,291.0,299.0,314.0,322.0,327.0,333.0,342.0,350.0,358.0,363.0,370.0,374.0,377.0,381.0,385.0,390.0,391.0,389.0,383.0,375.0,368.0,357.0,349.0,336.0,326.0,315.0,305.0,295.0,286.0,279.0
0,195.0,192.0,181.0,165.0,139.0,133.0,125.0,126.0,136.0,143.0,152.0,155.0,159.0,157.0,155.0,154.0,155.0,158.0,158.0,160.0,165.0,169.0,169.0,160.0,147.0,146.0,147.0,146.0,145.0,143.0
0,159.0,161.0,164.0,169.0,174.0,179.0,182.0,184.0,186.0,194.0,201.0,205.0,204.0,199.0,200.0,205.0,207.0,210.0,211.0,214.0,216.0,209.0,205.0,206.0,205.0,203.0,202.0,202.0,203.0,200.0
0,115.0,120.0,130.0,132.0,130.0,129.0,126.0,128.0,131.0,139.0,153.0,165.0,174.0,185.0,198.0,208.0,223.0,245.0,259.0,263.0,254.0,248.0,233.0,222.0,216.0,210.0,207.0,186.0,179.0,172.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,196.0,201.0,202.0,201.0,203.0,206.0,207.0,209.0,206.0,180.0,168.0,161.0,160.0,176.0,186.0,192.0,194.0,186.0,170.0,148.0,142.0,142.0,141.0,137.0,132.0,128.0,123.0,117.0,108.0,103.0
0,148.0,148.0,148.0,150.0,153.0,156.0,157.0,156.0,150.0,140.0,130.0,122.0,127.0,136.0,148.0,163.0,177.0,183.0,188.0,190.0,190.0,192.0,187.0,185.0,177.0,161.0,157.0,160.0,163.0,165.0
0,116.0,124.0,128.0,129.0,133.0,137.0,140.0,142.0,147.0,153.0,164.0,173.0,178.0,178.0,174.0,168.0,164.0,167.0,180.0,189.0,193.0,190.0,183.0,178.0,175.0,173.0,172.0,174.0,181.0,191.0
0,73.0,69.0,67.0,68.0,76.0,83.0,88.0,93.0,97.0,99.0,102.0,102.0,94.0,92.0,100.0,108.0,119.0,127.0,137.0,142.0,144.0,147.0,154.0,148.0,134.0,146.0,161.0,170.0,175.0,175.0


In [ ]:
meal_final = meal_data1.append(meal_data2)
meal_final["Label"] = 1
meal_final.reset_index(inplace=True, drop=True)
meal_final

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,Label
0,150.0,162.0,174.0,180.0,178.0,184.0,185.0,194.0,193.0,186.0,182.0,174.0,172.0,167.0,166.0,164.0,165.0,166.0,167.0,164.0,159.0,160.0,158.0,158.0,153.0,154.0,151.0,150.0,149.0,150.0,1
1,69.0,80.0,89.0,98.0,100.0,111.0,123.0,139.0,160.0,178.0,186.0,185.0,186.0,191.0,201.0,228.0,242.0,263.0,277.0,284.0,287.0,288.0,289.0,279.0,260.0,262.0,259.0,250.0,245.0,250.0,1
2,177.0,194.0,200.0,201.0,189.0,162.0,166.0,173.0,176.0,169.0,167.0,166.0,168.0,180.0,182.0,178.0,171.0,160.0,145.0,138.0,132.0,120.0,127.0,129.0,132.0,123.0,104.0,100.0,107.0,106.0,1
3,208.0,215.0,212.0,206.0,203.0,196.0,190.0,180.0,184.0,180.0,177.0,186.0,194.0,205.0,207.0,220.0,200.0,194.0,196.0,198.0,200.0,201.0,211.0,215.0,217.0,216.0,221.0,221.0,219.0,209.0,1
4,202.0,197.0,191.0,185.0,197.0,206.0,209.0,207.0,174.0,168.0,168.0,142.0,136.0,135.0,136.0,147.0,148.0,137.0,137.0,144.0,141.0,135.0,133.0,126.0,110.0,104.0,74.0,62.0,58.0,57.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
856,196.0,201.0,202.0,201.0,203.0,206.0,207.0,209.0,206.0,180.0,168.0,161.0,160.0,176.0,186.0,192.0,194.0,186.0,170.0,148.0,142.0,142.0,141.0,137.0,132.0,128.0,123.0,117.0,108.0,103.0,1
857,148.0,148.0,148.0,150.0,153.0,156.0,157.0,156.0,150.0,140.0,130.0,122.0,127.0,136.0,148.0,163.0,177.0,183.0,188.0,190.0,190.0,192.0,187.0,185.0,177.0,161.0,157.0,160.0,163.0,165.0,1
858,116.0,124.0,128.0,129.0,133.0,137.0,140.0,142.0,147.0,153.0,164.0,173.0,178.0,178.0,174.0,168.0,164.0,167.0,180.0,189.0,193.0,190.0,183.0,178.0,175.0,173.0,172.0,174.0,181.0,191.0,1
859,73.0,69.0,67.0,68.0,76.0,83.0,88.0,93.0,97.0,99.0,102.0,102.0,94.0,92.0,100.0,108.0,119.0,127.0,137.0,142.0,144.0,147.0,154.0,148.0,134.0,146.0,161.0,170.0,175.0,175.0,1


# Get no meal data

In [ ]:
meal_point_idx = meal_point.reset_index()
meal_point2_idx  = meal_point2.reset_index()

In [ ]:
def count_nomeal(df):
  count = 0
  count_h = 0 # number of 2 hr time periods
  no_meal_times = pd.DataFrame()
  for idx, row in df.iterrows():
    if idx >= len(df)-1:
      break
    time_between_meals =  (row['timestamp'] + timedelta(hours=0, minutes=-6)) - (df.at[idx+1,'timestamp'] + timedelta(hours=2, minutes=6))
    if time_between_meals > timedelta(hours=2, minutes=0):
      #print(time_between_meals)
      df.at[idx+1, "no_meal"] = time_between_meals
      current_hour_count = int(time_between_meals.seconds/3600)//2
      df.at[idx+1, "count_hours"] = current_hour_count
      count_h += current_hour_count
      count += 1
      no_meal_times = no_meal_times.append(df.iloc[idx+1])
      
  #print(count, count_h)
  return no_meal_times


In [ ]:
df_no_meal_1 = count_nomeal(meal_point_idx)
df_no_meal_1

,Alarm,BG Reading (mg/dL),BWZ Active Insulin (U),BWZ BG Input (mg/dL),BWZ Carb Input (grams),BWZ Carb Ratio (g/U),BWZ Correction Estimate (U),BWZ Estimate (U),BWZ Food Estimate (U),BWZ Insulin Sensitivity (mg/dL/U),BWZ Target High BG (mg/dL),BWZ Target Low BG (mg/dL),BWZ Unabsorbed Insulin Total (U),Basal Rate (U/h),Bolus Cancellation Reason,Bolus Duration (h:mm:ss),Bolus Number,Bolus Source,Bolus Type,Bolus Volume Delivered (U),Bolus Volume Selected (U),Date,Final Bolus Estimate,Index,Insulin Action Curve Time,Network Device Associated Reason,Network Device Disassociated Reason,Network Device Disconnected Reason,New Device Time,Prime Type,Prime Volume Delivered (U),Rewind,Scroll Step Size,Sensor Calibration BG (mg/dL),Sensor Calibration Rejected Reason,Suspend,Time,count_hours,index,no_meal,timestamp
1,NaN,NaN,NaN,0.0,15.0,12.0,0.00,1.2,1.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-02-12,NaN,129.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-10-28 02:30:55,2.0,129.0,0 days 04:32:50,2018-02-12 02:30:55
10,NaN,NaN,0.0,145.0,0.0,12.0,0.30,0.3,0.00,80.0,115.0,80.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-02-11,NaN,348.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-10-28 05:40:58,1.0,348.0,0 days 02:09:21,2018-02-11 05:40:58
11,NaN,NaN,NaN,0.0,40.0,8.0,0.00,5.0,5.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-02-10,NaN,432.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-10-28 19:46:28,3.0,432.0,0 days 07:42:30,2018-02-10 19:46:28
12,NaN,NaN,NaN,163.0,90.0,10.0,0.23,9.2,9.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-02-10,NaN,486.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-10-28 14:48:11,1.0,486.0,0 days 02:46:17,2018-02-10 14:48:11
13,NaN,NaN,0.0,193.0,0.0,12.0,0.90,0.9,0.00,80.0,115.0,80.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-02-10,NaN,576.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-10-28 03:13:32,4.0,576.0,0 days 09:22:39,2018-02-10 03:13:32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1029,NaN,NaN,0.3,129.0,47.0,9.0,0.00,5.2,5.20,65.0,140.0,90.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-07-27,NaN,21869.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-10-28 09:19:41,1.0,41164.0,0 days 02:25:08,2017-07-27 09:19:41
1031,NaN,NaN,0.0,86.0,60.0,9.5,-0.10,6.2,6.30,65.0,140.0,90.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-07-26,NaN,21919.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-10-28 19:15:06,4.0,41214.0,0 days 08:18:45,2017-07-26 19:15:06
1032,NaN,NaN,2.4,255.0,63.0,8.5,1.70,7.4,7.40,65.0,140.0,90.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-07-26,NaN,21966.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-10-28 12:48:41,2.0,41261.0,0 days 04:14:25,2017-07-26 12:48:41
1035,NaN,NaN,0.0,104.0,115.0,9.5,0.00,12.1,12.10,65.0,140.0,90.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-07-25,NaN,22052.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-10-28 18:31:40,6.0,41347.0,0 days 12:43:36,2017-07-25 18:31:40


In [ ]:
df_no_meal_2 = count_nomeal(meal_point2_idx)
df_no_meal_2

,Alarm,BG Reading (mg/dL),BWZ Active Insulin (U),BWZ BG Input (mg/dL),BWZ Carb Input (grams),BWZ Carb Ratio (g/U),BWZ Correction Estimate (U),BWZ Estimate (U),BWZ Food Estimate (U),BWZ Insulin Sensitivity (mg/dL/U),BWZ Target High BG (mg/dL),BWZ Target Low BG (mg/dL),BWZ Unabsorbed Insulin Total (U),Basal Rate (U/h),Bolus Cancellation Reason,Bolus Number,Bolus Source,Bolus Type,Bolus Volume Delivered (U),Bolus Volume Selected (U),Date,Final Bolus Estimate,Index,Insulin Action Curve Time,Network Device Associated Reason,Network Device Disassociated Reason,Network Device Disconnected Reason,New Device Time,Prime Type,Prime Volume Delivered (U),Rewind,Scroll Step Size,Sensor Calibration BG (mg/dL),Sensor Calibration Rejected Reason,Suspend,Temp Basal Amount,Temp Basal Duration (h:mm:ss),Temp Basal Type,Time,count_hours,index,no_meal,timestamp
1,NaN,NaN,0.8,358.0,0.0,10.0,2.30,1.5,0.00,92.0,140.0,120.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-03-07,NaN,6662.0,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00:07:36,2.0,98.0,0 days 04:54:01,2018-03-07 00:07:36
4,NaN,NaN,NaN,247.0,40.0,8.6,0.13,4.7,4.65,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-03-06,NaN,6779.0,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12:40:57,1.0,215.0,0 days 02:14:27,2018-03-06 12:40:57
7,NaN,NaN,NaN,166.0,0.0,9.2,0.28,0.2,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-03-05,NaN,6903.0,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22:43:51,3.0,339.0,0 days 07:46:13,2018-03-05 22:43:51
10,NaN,NaN,NaN,132.0,45.0,13.0,0.00,3.4,3.45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-03-05,NaN,6999.0,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11:52:19,1.0,435.0,0 days 03:15:10,2018-03-05 11:52:19
15,NaN,NaN,0.7,200.0,0.0,9.2,0.60,0.0,0.00,92.0,140.0,120.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-03-04,NaN,7055.0,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22:09:41,2.0,491.0,0 days 05:27:41,2018-03-04 22:09:41
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
719,NaN,NaN,0.0,206.0,33.0,8.6,0.70,4.5,3.80,92.0,140.0,120.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-09-07,NaN,17124.0,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12:54:00,1.0,22993.0,0 days 02:33:22,2017-09-07 12:54:00
722,NaN,NaN,0.8,178.0,0.0,9.2,0.40,0.0,0.00,92.0,140.0,120.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-09-06,NaN,17159.0,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22:40:09,3.0,23028.0,0 days 06:45:58,2017-09-06 22:40:09
725,NaN,NaN,0.0,124.0,33.0,8.6,0.00,3.8,3.80,92.0,140.0,120.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-09-06,NaN,17187.0,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13:22:26,1.0,23056.0,0 days 02:28:34,2017-09-06 13:22:26
726,NaN,NaN,0.0,191.0,60.0,13.0,0.50,5.1,4.60,92.0,140.0,120.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-09-06,NaN,17207.0,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,08:48:28,1.0,23076.0,0 days 02:21:58,2017-09-06 08:48:28


In [ ]:
def extract_no_meal_data(df_no_meal, df):
  no_meal_data = pd.DataFrame()
  time_increment = timedelta(hours=2, minutes=0)

  for idx, row in df_no_meal.iterrows():
    for i in range(1, int(row["count_hours"])+1):
      df_test = df[ (df["timestamp"] > (row["timestamp"] + timedelta(hours=i*2, minutes=0))) & (df["timestamp"] < (row["timestamp"] + time_increment + timedelta(hours=i*2, minutes=0)))]
      if len(df_test) != 24:
        #print(len(df_test), row["timestamp"])
        continue
      no_meal_data = no_meal_data.append([df_test["Sensor Glucose (mg/dL)"].to_list()])

  return no_meal_data

In [ ]:
no_meal_data1 = extract_no_meal_data(df_no_meal_1, df_cgm)
no_meal_data1.dropna(inplace=True)
no_meal_data1

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23
0,150.0,152.0,151.0,150.0,149.0,147.0,147.0,145.0,143.0,137.0,140.0,144.0,147.0,161.0,165.0,168.0,169.0,140.0,131.0,130.0,131.0,127.0,127.0,125.0
0,58.0,57.0,58.0,57.0,60.0,61.0,60.0,60.0,61.0,62.0,62.0,61.0,63.0,63.0,63.0,62.0,65.0,69.0,70.0,70.0,67.0,63.0,66.0,69.0
0,79.0,73.0,69.0,67.0,68.0,70.0,67.0,66.0,65.0,66.0,66.0,63.0,62.0,61.0,59.0,60.0,71.0,76.0,78.0,80.0,83.0,87.0,89.0,95.0
0,109.0,109.0,107.0,104.0,101.0,98.0,98.0,99.0,98.0,97.0,95.0,93.0,91.0,90.0,89.0,87.0,84.0,79.0,77.0,77.0,77.0,76.0,77.0,79.0
0,96.0,94.0,99.0,103.0,107.0,106.0,105.0,100.0,97.0,95.0,93.0,103.0,108.0,110.0,110.0,108.0,106.0,105.0,105.0,105.0,106.0,108.0,109.0,109.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,106.0,99.0,92.0,89.0,91.0,93.0,92.0,92.0,95.0,95.0,95.0,92.0,87.0,81.0,83.0,87.0,90.0,91.0,90.0,89.0,89.0,90.0,89.0,86.0
0,119.0,119.0,118.0,117.0,117.0,118.0,119.0,120.0,120.0,118.0,115.0,113.0,111.0,112.0,113.0,113.0,115.0,120.0,124.0,127.0,124.0,115.0,115.0,114.0
0,172.0,167.0,164.0,163.0,163.0,164.0,172.0,175.0,172.0,170.0,164.0,163.0,167.0,167.0,164.0,158.0,151.0,145.0,137.0,132.0,125.0,120.0,118.0,118.0
0,193.0,196.0,196.0,194.0,193.0,193.0,192.0,193.0,190.0,186.0,183.0,184.0,188.0,192.0,192.0,190.0,185.0,185.0,187.0,185.0,184.0,184.0,181.0,177.0


In [ ]:
no_meal_data2 = extract_no_meal_data(df_no_meal_2, df_cgm2)
no_meal_data2.dropna(inplace=True)
no_meal_data2

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23
0,296.0,297.0,295.0,293.0,292.0,292.0,292.0,291.0,289.0,285.0,282.0,281.0,280.0,280.0,280.0,281.0,284.0,287.0,286.0,285.0,280.0,277.0,279.0,284.0
0,271.0,271.0,272.0,272.0,273.0,274.0,275.0,275.0,274.0,273.0,275.0,282.0,289.0,292.0,292.0,286.0,282.0,283.0,282.0,282.0,285.0,288.0,292.0,295.0
0,200.0,195.0,189.0,179.0,165.0,157.0,152.0,146.0,140.0,131.0,127.0,125.0,125.0,125.0,123.0,122.0,121.0,118.0,116.0,113.0,110.0,108.0,105.0,107.0
0,112.0,111.0,111.0,110.0,110.0,108.0,107.0,106.0,105.0,103.0,101.0,100.0,100.0,101.0,103.0,105.0,103.0,102.0,102.0,102.0,103.0,103.0,104.0,105.0
0,120.0,118.0,113.0,110.0,110.0,110.0,110.0,111.0,108.0,106.0,110.0,112.0,114.0,115.0,112.0,111.0,111.0,111.0,111.0,112.0,112.0,112.0,112.0,112.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,188.0,190.0,187.0,184.0,183.0,184.0,189.0,191.0,194.0,198.0,207.0,209.0,205.0,205.0,215.0,224.0,232.0,235.0,230.0,218.0,200.0,192.0,188.0,190.0
0,98.0,89.0,82.0,76.0,73.0,81.0,86.0,87.0,89.0,92.0,93.0,94.0,94.0,99.0,105.0,110.0,125.0,137.0,141.0,141.0,139.0,141.0,145.0,147.0
0,199.0,197.0,194.0,189.0,185.0,182.0,179.0,177.0,174.0,170.0,165.0,159.0,153.0,147.0,139.0,129.0,122.0,120.0,122.0,126.0,128.0,124.0,121.0,118.0
0,170.0,170.0,170.0,170.0,171.0,175.0,180.0,187.0,194.0,199.0,202.0,204.0,205.0,205.0,204.0,204.0,205.0,205.0,205.0,204.0,204.0,204.0,204.0,202.0


In [ ]:
no_meal_final = no_meal_data1.append(no_meal_data2)
no_meal_final["Label"] = 0
no_meal_final.reset_index(inplace=True, drop=True)
no_meal_final

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,Label
0,150.0,152.0,151.0,150.0,149.0,147.0,147.0,145.0,143.0,137.0,140.0,144.0,147.0,161.0,165.0,168.0,169.0,140.0,131.0,130.0,131.0,127.0,127.0,125.0,0
1,58.0,57.0,58.0,57.0,60.0,61.0,60.0,60.0,61.0,62.0,62.0,61.0,63.0,63.0,63.0,62.0,65.0,69.0,70.0,70.0,67.0,63.0,66.0,69.0,0
2,79.0,73.0,69.0,67.0,68.0,70.0,67.0,66.0,65.0,66.0,66.0,63.0,62.0,61.0,59.0,60.0,71.0,76.0,78.0,80.0,83.0,87.0,89.0,95.0,0
3,109.0,109.0,107.0,104.0,101.0,98.0,98.0,99.0,98.0,97.0,95.0,93.0,91.0,90.0,89.0,87.0,84.0,79.0,77.0,77.0,77.0,76.0,77.0,79.0,0
4,96.0,94.0,99.0,103.0,107.0,106.0,105.0,100.0,97.0,95.0,93.0,103.0,108.0,110.0,110.0,108.0,106.0,105.0,105.0,105.0,106.0,108.0,109.0,109.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1378,188.0,190.0,187.0,184.0,183.0,184.0,189.0,191.0,194.0,198.0,207.0,209.0,205.0,205.0,215.0,224.0,232.0,235.0,230.0,218.0,200.0,192.0,188.0,190.0,0
1379,98.0,89.0,82.0,76.0,73.0,81.0,86.0,87.0,89.0,92.0,93.0,94.0,94.0,99.0,105.0,110.0,125.0,137.0,141.0,141.0,139.0,141.0,145.0,147.0,0
1380,199.0,197.0,194.0,189.0,185.0,182.0,179.0,177.0,174.0,170.0,165.0,159.0,153.0,147.0,139.0,129.0,122.0,120.0,122.0,126.0,128.0,124.0,121.0,118.0,0
1381,170.0,170.0,170.0,170.0,171.0,175.0,180.0,187.0,194.0,199.0,202.0,204.0,205.0,205.0,204.0,204.0,205.0,205.0,205.0,204.0,204.0,204.0,204.0,202.0,0


# save final csv

In [ ]:
final_data = meal_final.append(no_meal_final)
final_data.reset_index(inplace=True, drop=True)
final_data

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,Label
0,150.0,162.0,174.0,180.0,178.0,184.0,185.0,194.0,193.0,186.0,182.0,174.0,172.0,167.0,166.0,164.0,165.0,166.0,167.0,164.0,159.0,160.0,158.0,158.0,153.0,154.0,151.0,150.0,149.0,150.0,1
1,69.0,80.0,89.0,98.0,100.0,111.0,123.0,139.0,160.0,178.0,186.0,185.0,186.0,191.0,201.0,228.0,242.0,263.0,277.0,284.0,287.0,288.0,289.0,279.0,260.0,262.0,259.0,250.0,245.0,250.0,1
2,177.0,194.0,200.0,201.0,189.0,162.0,166.0,173.0,176.0,169.0,167.0,166.0,168.0,180.0,182.0,178.0,171.0,160.0,145.0,138.0,132.0,120.0,127.0,129.0,132.0,123.0,104.0,100.0,107.0,106.0,1
3,208.0,215.0,212.0,206.0,203.0,196.0,190.0,180.0,184.0,180.0,177.0,186.0,194.0,205.0,207.0,220.0,200.0,194.0,196.0,198.0,200.0,201.0,211.0,215.0,217.0,216.0,221.0,221.0,219.0,209.0,1
4,202.0,197.0,191.0,185.0,197.0,206.0,209.0,207.0,174.0,168.0,168.0,142.0,136.0,135.0,136.0,147.0,148.0,137.0,137.0,144.0,141.0,135.0,133.0,126.0,110.0,104.0,74.0,62.0,58.0,57.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2239,188.0,190.0,187.0,184.0,183.0,184.0,189.0,191.0,194.0,198.0,207.0,209.0,205.0,205.0,215.0,224.0,232.0,235.0,230.0,218.0,200.0,192.0,188.0,190.0,NaN,NaN,NaN,NaN,NaN,NaN,0
2240,98.0,89.0,82.0,76.0,73.0,81.0,86.0,87.0,89.0,92.0,93.0,94.0,94.0,99.0,105.0,110.0,125.0,137.0,141.0,141.0,139.0,141.0,145.0,147.0,NaN,NaN,NaN,NaN,NaN,NaN,0
2241,199.0,197.0,194.0,189.0,185.0,182.0,179.0,177.0,174.0,170.0,165.0,159.0,153.0,147.0,139.0,129.0,122.0,120.0,122.0,126.0,128.0,124.0,121.0,118.0,NaN,NaN,NaN,NaN,NaN,NaN,0
2242,170.0,170.0,170.0,170.0,171.0,175.0,180.0,187.0,194.0,199.0,202.0,204.0,205.0,205.0,204.0,204.0,205.0,205.0,205.0,204.0,204.0,204.0,204.0,202.0,NaN,NaN,NaN,NaN,NaN,NaN,0


In [ ]:
final_data.to_csv("meal_nomeal_data.csv", index=False)

# Feature extraction

In [ ]:
import scipy
import scipy.signal
import scipy.fftpack

In [ ]:
#velocity
def get_velocity(df):
  df_vel = pd.DataFrame()
  vel_feat = pd.DataFrame()
  #print(df.shape)
  for i in range(0, df.shape[1] - 1):
    df_vel['vel' + str(i)] = (df.iloc[:, i + 1] - df.iloc[:, i])

  vel_feat["vel_min"] = df_vel.min(axis=1, skipna=True)
  vel_feat["vel_mean"] = df_vel.mean(axis=1, skipna=True)
  vel_feat["vel_max"] = df_vel.max(axis=1, skipna=True)

  return vel_feat

In [ ]:
# PSD / periodogram
def get_psd_row(df):
  f, psd_dat = scipy.signal.periodogram(df.dropna())
  psd_row = [psd_dat[0:5].mean(), psd_dat[5:10].mean(), psd_dat[10:16].mean()]
  return psd_row


def get_psd(df):
  PSD = pd.DataFrame()
  PSD['vals'] = df.apply(lambda row: get_psd_row(row), axis=1)
  PSD_feat = pd.DataFrame(PSD.vals.tolist(), columns=['PSD_5', 'PSD_10', 'PSD_15'])

  return PSD_feat

In [ ]:
# IQR / inter quartile range
def get_iqr_row(df):
  iqrDat = scipy.stats.iqr(df.dropna())
  return iqrDat


def get_iqr(df):
  IQR_feat = pd.DataFrame()
  IQR_feat['IQR'] = df.apply(lambda x: get_iqr_row(x), axis=1)
  return IQR_feat

In [ ]:
# entropy
def get_entropy_row(df):
  df_counts = df.dropna().value_counts()
  entropy = scipy.stats.entropy(df_counts)
  return entropy


def get_entropy(df):
  entropy_feat = pd.DataFrame()
  entropy_feat['Entropy'] = df.apply(lambda x: get_entropy_row(x), axis=1)
  return entropy_feat


In [ ]:
# Fourier Transform
def get_fft_row(df):
  #print(df.to_list())
  
  FFT_vals = abs(scipy.fftpack.fft(df.dropna().to_list()))
  FFT_vals.sort()
  return np.flip(FFT_vals)[0:6]


def get_fft(df):
  FFT = pd.DataFrame()
  FFT['vals'] = df.apply(lambda row: get_fft_row(row), axis=1)
  FFT_feat = pd.DataFrame(FFT.vals.tolist(), columns=['FFT_1', 'FFT_2', 'FFT_3', 'FFT_4', 'FFT_5', 'FFT_6'])
  return FFT_feat

In [ ]:
# Time difference between CGM max and CGM min
# def get_timediff(df):
  

In [ ]:
########### features for all data

In [ ]:
X = pd.concat( [get_velocity(final_data.drop(columns=["Label"])),
get_psd(final_data.drop(columns=["Label"])),
get_iqr(final_data.drop(columns=["Label"])),
get_entropy(final_data.drop(columns=["Label"])),
get_fft(final_data.drop(columns=["Label"]))], axis = 1)
X

,vel_min,vel_mean,vel_max,PSD_5,PSD_10,PSD_15,IQR,Entropy,FFT_1,FFT_2,FFT_3,FFT_4,FFT_5,FFT_6
0,-8.0,0.000000,12.0,1000.744306,22.864991,5.686697,19.00,3.060287,5015.0,240.345182,240.345182,126.233819,126.233819,31.506578
1,-19.0,6.241379,27.0,27974.786169,1562.733444,627.228101,117.25,3.308778,6059.0,1345.632057,1345.632057,394.763112,394.763112,322.894884
2,-27.0,-2.448276,17.0,4987.790521,205.910648,91.226804,47.00,3.308778,4642.0,470.640332,470.640332,259.940832,259.940832,215.545678
3,-20.0,0.034483,13.0,909.883038,57.943776,16.527655,19.75,3.123939,6081.0,207.934591,207.934591,111.347548,111.347548,109.448473
4,-33.0,-5.000000,12.0,9782.821167,1117.294574,434.481327,48.75,3.170148,4366.0,600.391832,600.391832,537.011660,537.011660,216.785819
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2239,-18.0,0.086957,10.0,1221.516938,8.309494,2.900392,21.75,2.947005,4838.0,243.721487,243.721487,83.641712,83.641712,76.308041
2240,-9.0,2.130435,15.0,2801.714309,92.580567,37.286318,49.00,2.925203,2564.0,370.758046,370.758046,156.902956,156.902956,56.709788
2241,-10.0,-3.521739,4.0,3512.987596,254.019587,130.640806,54.25,3.120292,3719.0,417.395962,417.395962,123.655635,123.655635,103.319892
2242,-2.0,1.391304,7.0,928.864941,39.185547,16.568632,25.25,1.986386,4643.0,210.922914,210.922914,96.779614,96.779614,31.606961


In [ ]:
# checking is any Nan Values
if not X.isnull().sum().sum() == 0:
  print("NAN VALUES IN TRAIN DATA")

In [ ]:
Y = final_data["Label"]
Y

0       1
1       1
2       1
3       1
4       1
       ..
2239    0
2240    0
2241    0
2242    0
2243    0
Name: Label, Length: 2244, dtype: int64

# Training model

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
import pickle

In [ ]:
from sklearn.model_selection import train_test_split
from pprint import pprint

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
model_params = {
    'n_estimators': [50, 150, 250],
    'max_features': ['sqrt', 0.25, 0.5, 0.75, 1.0],
    'min_samples_split': [2, 4, 6]
}

In [ ]:
# create random forest classifier model
rf_model = RandomForestClassifier(random_state=1)

# set up grid search meta-estimator
clf = GridSearchCV(rf_model, model_params, cv=5)

# train the grid search meta-estimator to find the best model
model = clf.fit(X, Y)

# print winning set of hyperparameters
#pprint(model.best_estimator_.get_params())

In [ ]:
# save the model to disk
filename = 'model.pkl'
pickle.dump(model, open(filename, 'wb'))

In [ ]:
## traininf on 80%
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42, stratify=Y, shuffle=True)

# create random forest classifier model
rf_model80 = RandomForestClassifier(random_state=42)

# set up grid search meta-estimator
clf80 = GridSearchCV(rf_model80, model_params, cv=5)

# train the grid search meta-estimator to find the best model
model80 = clf80.fit(X_train, y_train)

# print winning set of hyperparameters
#pprint(model80.best_estimator_.get_params())

In [ ]:
y_pred = model80.predict(X_test)

In [ ]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.92      0.99      0.96       277
           1       0.99      0.87      0.92       172

    accuracy                           0.94       449
   macro avg       0.95      0.93      0.94       449
weighted avg       0.95      0.94      0.94       449



In [ ]:
# save the model to disk
filename = 'model_80.pkl'
pickle.dump(model80, open(filename, 'wb'))